<center><a href="https://www.nvidia.cn/training/"><img src="https://dli-lms.s3.amazonaws.com/assets/general/DLI_Header_White.png" width="400" height="186" /></a></center>

# 思维链提示工程（Chain-of-Thought Prompting）

In [ ]:
from videos.walkthroughs import walkthrough_34 as walkthrough

In [ ]:
walkthrough()

在这个 notebook 中，您将了解一种非常著名的提示技巧，叫做思维链提示。

---

## 目标

完成这个 notebook 后，您将：

- 学习思维链提示
- 遇到并理解 LLM 的幻觉

---

## 导入

In [ ]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

---

## 创建模型实例

In [ ]:
base_url = 'http://llama:8000/v1'
model = 'meta/llama-3.1-8b-instruct'
llm = ChatNVIDIA(base_url=base_url, model=model, temperature=0)

---

## LLM 直接给出结论

由于 LLM 是为了生成文本流中接下来最可能的内容而设计的，因此它们在需要复杂推理的场景中“直接给出结论”（jump to conclusions）并不奇怪。

考虑以下例子，它让 LLM 做乘法。在调用 LLM 之前，我们先用 Python 做出正确答案。

In [ ]:
345*888

In [ ]:
print(llm.invoke('What is 345 * 888?').content)

这个答案是错误的，但据我们了解，LLM 主要是参考根据训练数据生成最可能的响应，所以回答不对并不出乎意料。

未来几年可能会发生重大变化，但目前而言，尽管常用“人工**智能**”等术语，LLM 并不太适合逐步推理，至少在没有额外用户支持的情况下。处于其设计方式，LLM 更适合直接跳到结论。

---

## 幻觉（Hallucination）

LLM 生成不正确内容的现象，往往让外部观察者感觉它很“自信”，我们把这种现象称为**幻觉**。处于其设计机制，LLM 宁可说一些不正确的内容（而它自己并不知道）也不会什么都不说。虽然 LLM 会随着时间的推移减少幻觉，但目前它仍然是 LLM 生成的一个特性。

关于 LLM 幻觉，您需要牢记：
1. 所有 LLM 都会出现幻觉。
2. 您需要对自己应用中 LLM 生成的内容负责。

---

## 思维链提示工程

回到这个 notebook 的主要内容，虽然 LLM 并不适合长时间的深思熟虑推理，但我们可以利用一些技巧来引导 LLM 在执行任务时采用循序渐进的方式。其中一种技巧就是思维链。

思维链提示在 2023 年被提出，现在仍是最受欢迎的提示技巧之一。它通过鼓励 LLM 将复杂问题分解为中间步骤，得以支持复杂的推理能力。

以下是思维链提示论文中的一个示例。

![思维链](images/Chain.png)

正如您在右侧看到的，通过让 LLM 将复杂任务分解为更小的子任务，模型有了“思考的时间”，这使它在需要复杂推理的任务中表现得更好。

---

## 思维链乘法

我们来试试 3 位数乘法的思维链提示。从一个乘法提示开始。

In [ ]:
example_problem = 'What is 678 * 789?'

接下来，我们提供一个逐步解决问题的思维链示例。

In [ ]:
example_cot = '''\
Let me break this down into steps. First I'll break down 789 into hundreds, tens, and ones:

789 -> 700 + 80 + 9

Next I'll multiply 678 by each of these values, storing the intermediate results:

678 * 700 -> 678 * 7 * 100 -> 4746 * 100 -> 474600

My first intermediate result is 474600.

678 * 80 -> 678 * 8 * 10 -> 5424 * 10 -> 54240

My second intermediate result is 54240.

678 * 9 -> 6102

My third intermediate result is 6102.

My three intermediate results are 474600, 54240, and 6102.

Adding the first two intermediate results I get 474600 + 54240 -> 528840.

Adding 528840 to the last intermediate result I get 528840 + 6102 -> 534942

The final result is 534942.
'''

通过示例问题和示例思维链响应，我们可以构建一个单样本提示模板。

In [ ]:
multiplication_template = ChatPromptTemplate.from_messages([
    ('human', example_problem),
    ('ai', example_cot),
    ('human', '{long_multiplication_prompt}')
])

接下来我们将在一个简单的链中使用这个模板。

In [ ]:
multiplication_chain = multiplication_template | llm | StrOutputParser()

现在再看看 LLM 对乘法的表现是否有所提升。

In [ ]:
print(multiplication_chain.invoke('What is 345 * 888?'))

In [ ]:
345*888

正如您所见，LLM 遵循了我们思维链提示中的示例，分步解决了问题，在这种情况下生成了正确的响应。

---

## 零样本思维链提示

思维链提示可能会变得非常冗长，正如我们之前提到的，您不必害怕通过很长的提示词来达到目标。

话虽如此，利用 LLM 思维链还有更优雅的方式。其中一种变体叫做[零样本思维链](https://arxiv.org/abs/2205.11916)。这种提示技术只需在提示中添加“让我们逐步思考”（Let's think step by step）就可以了，无需提供冗长的思维链示例。

让我们构建一个新的提示模板来尝试在长乘法上使用零样本思维链，按照论文中的示例，仅提供“让我们逐步思考”这句话。

In [ ]:
zero_shot_cot_prompt = ChatPromptTemplate([
    ("human", "{long_multiplication_prompt} Let's think step by step.")
])

In [ ]:
zero_shot_multiplication_chain = zero_shot_cot_prompt | llm | StrOutputParser()

In [ ]:
print(zero_shot_multiplication_chain.invoke('What is 345 * 888?'))

In [ ]:
345*888

对于我们的三位数长乘法问题，简单提示模型去逐步思考的效果，跟更冗长的思维链提示一样好用。

---

## 实践中的思维链提示

和许多提示工程的其它方面一样，考虑到现在和未来可用的多种 LLM，确切地辨别什么时候以及如何使用思维链提示是相对困难的。不过，我们可以提供一些好的通用指南供您参考。

- 迭代开发提示。先从简单开始，当您认为零样本思维链可能有帮助时尝试使用它，必要时再扩展到更详细的基于示例的思维链提示。
- 考虑在面对不太适合 LLM 的任务（比如数学）时使用外部的非 LLM 工具。我们将在后面的课程中讨论工具的使用，但一般来说，LLM 都能表现得很出色，虽然并不一定适合每一个任务。想一想长乘法的例子，简单的 Python 是多么快速、可靠和有效。不要陷入*“如果你手里只有一把锤子，所有东西看起来都像钉子。”的陷阱，LLM 只是您在构建 LLM 应用时的众多工具之一。

---

## 练习：使用 LLM 解决一个文字问题

在这个练习中，使用您在本 notebook 中学到的内容，从 LLM 得到以下文字问题的正确响应。

In [ ]:
word_problem = """Michael's car travels at 40 miles per hour. He is driving from 1 PM to 4 PM and then \
travels back at a rate of 25 miles per hour due to heavy traffic. How long in \
terms of minutes did it take him to get back?"""

这是正确答案。

迈克尔以每小时 40 英里行驶了 3 个小时，这意味着他行驶了 120 英里。

以每小时 25 英里的速度回程需要 4.8 小时（120 英里 / 25 mph），这相当于 **288** 分钟（4.8 小时 * 60 分钟/小时）。

所以正确答案是 **288**。

如果您卡住了，可以查看下面的*参考答案*。

### 您的代码

### 参考答案

解决这个问题的方法有很多，这里我们选择用系统消息，鼓励 LLM 始终将工作分解为更小的任务并展示过程，然后将标准的零样本思维链提示附加到人类用户输入的内容上。

In [ ]:
template = ChatPromptTemplate.from_messages([
    ('system', 'You are an expert word problem solver. You always break your problem down into smaller tasks and show your work.'),
    ('human', '{prompt}\n\nLet\'s think step by step.')
])

In [ ]:
chain = template | llm | StrOutputParser()

In [ ]:
print(chain.invoke(word_problem))

---

## 总结

在这个 notebook 中，您遇到了 LLM 的幻觉，体会到 LLM 自然地倾向于草率得出结论。学习了几种思维链提示技巧，帮助 LLM 顺利完成需要逐步推理的任务。

下一个 notebook 您将学习如何使用现在掌握的提示工程技巧制作聊天机器人。